# Код Хэмминга #
В данной работе будет рассмотрен кодирование и декодирование сообщения, нахождение и справление одиночной ошибки.

In [1]:
import numpy as np

message = np.array([1,0,1,0])
m = len(message)


Нахождение k - минимального количества проверочных бит по правилу 2^k >= k+m+1, где m - длина сообщения.

In [2]:
k = 0
while 2**k < k+m+1:
   k+=1

## Построение порождающей матрицы ##
Порождающая матрица *generator_matrix* будет иметь вид [I | A], где I - единичная матрица размера m x m. 

In [3]:
def get_indexes(n):
    indexes = list(range(1,m+k+1))[2**n:2*2**n-1]
    for i in range(3*2**n, m+k+1, 2*2**n):
        indexes = np.concatenate((indexes, list(range(1,m+k+1))[i-1:i+2**n-1]))
    return list(map(int, indexes))

In [4]:
A = np.zeros([m, k])
for j in range(k):
    indexes = list(map(lambda x: x - len(format(x, "b"))-1, get_indexes(j)))
    for l in indexes:
        A[l][j] = 1

In [5]:
generator_matrix = np.concatenate((np.eye(m), A), axis=1)
generator_matrix

array([[1., 0., 0., 0., 1., 1., 0.],
       [0., 1., 0., 0., 1., 0., 1.],
       [0., 0., 1., 0., 0., 1., 1.],
       [0., 0., 0., 1., 1., 1., 1.]])

In [6]:
syndrome_matrix = np.concatenate((A.transpose(), np.eye(k)), axis=1)
syndrome_matrix

array([[1., 1., 0., 1., 1., 0., 0.],
       [1., 0., 1., 1., 0., 1., 0.],
       [0., 1., 1., 1., 0., 0., 1.]])

In [7]:
hamming_message = message.dot(generator_matrix)
hamming_message = np.array(list(map(lambda x: x%2, hamming_message)))
hamming_message

array([1., 0., 1., 0., 1., 0., 1.])

In [8]:
error_message = np.array([1,1,1,0,1,0,1])
syndrome = error_message.dot(syndrome_matrix.transpose())
syndrome = list(map(lambda x: x%2, syndrome))
syndrome

[1.0, 0.0, 1.0]